# `FieldSelector`

The `FieldSelector` class is used internally to extract, re-order, and/or rename fields in the item stream produced by a custom generator when items are exported (e.g. to a CSV file or a SQL table).

In [1]:
from tohu.tohu_items_class import make_tohu_items_class
from tohu.field_selector import FieldSelector

To illustrate how the `FieldSelector` class works let's manually define a list of tohu items to which we can apply different field selectors below.

In [2]:
Quux = make_tohu_items_class("Quux", ["aa", "bb", "cc"])

items = [
    Quux(111, "foo", "AAA"),
    Quux(222, "bar", "BBB"),
    Quux(333, "baz", "CCC"),
    Quux(444, "quux", "DDD"),
]

items

[Quux(aa=111, bb='foo', cc='AAA'),
 Quux(aa=222, bb='bar', cc='BBB'),
 Quux(aa=333, bb='baz', cc='CCC'),
 Quux(aa=444, bb='quux', cc='DDD')]

## No explicit field names: passing fields straight through without altering them

If the `fields` argument is not specified, the `FieldSelector` acts like the identity function, i.e., it passes items straight through without altering them. While this may not seem very useful, this case is important for consistency to avoid special case distinctions in the code when the user simply wants to export all fields of a custom generator without modification.

Note that the output values are ordinary dictionaries instead of tohu items. This is because tohu items are internally constructed as [attrs](https://www.attrs.org/en/stable/) classes but we want to allow renaming fields and the output names may not be valid attribute names (e.g. if they contain whitespace such as the name `"Column X"`).

In [9]:
fs = FieldSelector(Quux, fields=None)

list(fs(items))

[{'aa': 111, 'bb': 'foo', 'cc': 'AAA'},
 {'aa': 222, 'bb': 'bar', 'cc': 'BBB'},
 {'aa': 333, 'bb': 'baz', 'cc': 'CCC'},
 {'aa': 444, 'bb': 'quux', 'cc': 'DDD'}]

## Passing a list of field names: extracting fields and/or changing their order

If the `fields` argument is a list of field names, those fi

In [5]:
fs = FieldSelector(Quux, fields=["cc", "aa"])

In [6]:
list(fs(items))

[{'cc': 'AAA', 'aa': 111},
 {'cc': 'BBB', 'aa': 222},
 {'cc': 'CCC', 'aa': 333},
 {'cc': 'DDD', 'aa': 444}]

## Passing a dictionary: renaming fields

In [7]:
fs = FieldSelector(Quux, {"Column X": "bb", "Column Y": "aa"})

In [8]:
list(fs(items))

[{'Column X': 'foo', 'Column Y': 111},
 {'Column X': 'bar', 'Column Y': 222},
 {'Column X': 'baz', 'Column Y': 333},
 {'Column X': 'quux', 'Column Y': 444}]

## *TODO:* Error for non-existing fields

In [9]:
import pytest
#from tohu.field_selector import InvalidFieldError

In [10]:
# with pytest.raises(InvalidFieldError, match="Invalid fields: \['xx', 'yy'\]. Fields must be a subset of: \['aa', 'bb', 'cc'\]"):
#     fs = FieldSelector(Quux, fields=["aa", "xx", "bb", "yy"])

## *TODO:* Extracting nested fields

In [11]:
Quux = make_tohu_items_class("Quux", ["aa", "bb", "cc"])
Foobar = make_tohu_items_class("Foobar", ["xx", "yy"])

items = [
    Quux(111, "foo", Foobar("AAA", 10)),
    Quux(222, "bar", Foobar("BBB", 20)),
    Quux(333, "baz", Foobar("CCC", 30)),
    Quux(444, "quux", Foobar("DDD", 40)),
]

items

[Quux(aa=111, bb='foo', cc=Foobar(xx='AAA', yy=10)),
 Quux(aa=222, bb='bar', cc=Foobar(xx='BBB', yy=20)),
 Quux(aa=333, bb='baz', cc=Foobar(xx='CCC', yy=30)),
 Quux(aa=444, bb='quux', cc=Foobar(xx='DDD', yy=40))]

*TODO:* Ensure we can deal with nested fields; also ensure we raised an error if nested fields have the wrong names!

In [12]:
#fs = FieldSelector({"Column X": "cc.xx", "Column Y": "cc.yy", "Column Z": "aa"})